<a href="https://colab.research.google.com/github/Ndifreke000/Machine-Learning-Algorithms/blob/main/Book_Recommendation_System_Report_and_ImplementationUsingKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-10-05 13:28:37--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  76.1MB/s    in 0.3s    

2023-10-05 13:28:38 (76.1 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [7]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Load the data
books = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

# Data cleaning and filtering
user_counts = ratings['user_id'].value_counts()
book_counts = ratings['book_id'].value_counts()

# Filter users with at least 200 ratings and books with at least 100 ratings
filtered_users = user_counts[user_counts >= 200].index
filtered_books = book_counts[book_counts >= 100].index

filtered_ratings = ratings[(ratings['user_id'].isin(filtered_users)) & (ratings['book_id'].isin(filtered_books))]

# Create a pivot table for Nearest Neighbors
pivot_table = pd.pivot_table(filtered_ratings, values='rating', index='user_id', columns='book_id', fill_value=0)

# Convert the pivot table to a sparse matrix
sparse_matrix = csr_matrix(pivot_table.values)

# Fit the Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model_knn.fit(sparse_matrix)

# Function to get book recommendations
def get_recommends(book_title):
    # Find the book_id for the given title
    book_info = books[books['title'] == book_title]

    if book_info.empty:
        return f"Book with title '{book_title}' not found."

    book_id = book_info['book_id'].values[0]

    # Get the index of the book in the sparse matrix
    book_index = pivot_table.columns.get_loc(book_id)

    # Get the distances and indices of the nearest neighbors
    distances, indices = model_knn.kneighbors(pivot_table.iloc[:, book_index].values.reshape(1, -1), n_neighbors=6)

    # Create a list of recommended books with distances
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        book_id = pivot_table.columns[indices.flatten()[i]]
        book_title = books[books['book_id'] == book_id]['title'].values[0]
        distance = distances.flatten()[i]
        recommended_books.append([book_title, distance])

    return [book_title, recommended_books]

# Testing the function
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")


"Book with title 'The Queen of the Damned (Vampire Chronicles (Paperback))' not found."

In [8]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [9]:
def get_recommends(book_title):
    # Find the book_id for the given title
    book_match = books[books['title'] == book_title]

    if book_match.empty:
        return f"Book '{book_title}' not found in the dataset."

    book_id = book_match['book_id'].values[0]

    # Get the index of the book in the sparse matrix
    book_index = pivot_table.columns.get_loc(book_id)

    # Get the distances and indices of the nearest neighbors
    distances, indices = model_knn.kneighbors(pivot_table.iloc[:, book_index].values.reshape(1, -1), n_neighbors=6)

    # Create a list of recommended books with distances
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        book_id = pivot_table.columns[indices.flatten()[i]]
        book_title = books[books['book_id'] == book_id]['title'].values[0]
        distance = distances.flatten()[i]
        recommended_books.append([book_title, distance])

    return [book_title, recommended_books]

# Testing the function
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)


Book 'Where the Heart Is (Oprah's Book Club (Paperback))' not found in the dataset.
